#Student Name: 
#ECE 595 Machine Learning II
#Project 3: GAN - Student Code

In [1]:
#Import necessary packages
import numpy as np
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model,Sequential
from keras.datasets import mnist
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape
from keras.optimizers import Adam
from keras.models import load_model
import matplotlib.pyplot as plt

Using TensorFlow backend.


#Part 1: Implementing the GAN

In [1]:
#Load MNIST data and normalize to [-1, 1]
# Fill this in
(data_train, _), (_, _) = mnist.load_data()
data_train = (data_train - 127.5)/127.5

data_train = np.expand_dims(data_train, axis=3)

# The D-dimensional noise vector length
latent_dim = 100
data_dim = 28 * 28 * 1

img_shape=(28,28,1)

# Optimizer for discriminator, which will have a higher learning rate than adversarial model
def adam_optimizer():
    # FILL THIS IN
    return Adam(lr=0.0002, beta_1=0.5)

def gan_optimizer():
    return Adam(lr=0.001)


# Genrerator model
def create_generator():
    # FILL THIS IN
    generator=Sequential()
    generator.add(Dense(256, input_dim=latent_dim))
    generator.add(LeakyReLU(0.5))
    generator.add(Dense(512))
    generator.add(LeakyReLU(0.5))
    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.5))
    generator.add(Dense(data_dim, activation='tanh'))
    
    return generator

# Discriminator model
def create_discriminator():
    # FILL THIS IN
    discriminator = Sequential()
    discriminator.add(Dense(1024, input_dim=data_dim))
    discriminator.add(LeakyReLU(0.5))
    discriminator.add(Dropout(0.3))
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.5))
    discriminator.add(Dropout(0.3))
    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.5))
    discriminator.add(Dense(units=1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', optimizer = adam_optimizer(), metrics=['accuracy'])
    return discriminator

# Create adversarial model
def create_gan(discriminator, generator):
    # FILL THIS IN
    discriminator.trainable = False
    gan_input = Input(shape=(latent_dim,))
    x=generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer = gan_optimizer(), metrics = ['accuracy'])
    return gan

# Creating GAN
generator = create_generator()
discriminator = create_discriminator()
gan = create_gan(discriminator, generator)

# Model and training parameters
#ASSIGN VALUES TO THE FOLLOWING VARIABLES
epochs = 5000
batch_size = 128
sample_interval = 500

# Array to save training history
training_meta_data = np.zeros([epochs, 4])

# Training the GAN
for e in range(1, epochs+1):

    # Generate random noise as input
    # FILL THIS IN
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    
    # Generate fake MNIST images from generated noise
    # FILL THIS IN
    fake_images = generator.predict(noise)
    fake_images = np.reshape(fake_images, (batch_size, 28, 28, 1))

    # Get a random set of real MNIST images
    # FILL THIS IN
    idx= np.random.randint(0, data_train.shape[0], batch_size)
    real_images = data_train[idx]

    # Concatenate real and fake images into a single array (or batch)
    # FILL THIS IN
    
    data_total = np.concatenate((real_images, fake_images), axis = 0)
    data_total = np.reshape(data_total, (-1, data_dim))

    # Assign training labels (assign high probability, but not 1, to real images)
    # FILL THIS IN
    labels_real = 0.98*np.ones((batch_size, 1))
    labels_fake = 0.02*np.zeros((batch_size, 1))
    labels_dis = np.concatenate((labels_real, labels_fake),axis = 0)

    # Allow discriminator parameters to be updated
    # FILL THIS IN
    discriminator.trainable = True

    # Train discriminator on batch of real and fake images. Assign loss and accuracy to variable
    # FILL THIS IN
    dis_training_meta_data = discriminator.fit(data_total, labels_dis, epochs=epochs, batch_size=batch_size, shuffle=True).history
    d_loss = [dis_training_meta_data['loss'][-1], dis_training_meta_data['accuracy'][-1]]

    
    # Train adversarial model and try to fool discriminator (with incorrect label) 
    # by generating a new batch of noise and assign them labels of real data
    # FILL THIS IN
    noise2 = np.random.normal(0, 1, (batch_size, latent_dim))
    labels_generator = np.ones((batch_size, 1))

    # Keep discriminator weights constant while training generator
    # FILL THIS IN
    discriminator.trainable = False

    # Train GAN (without updating discriminator weights) on new batch of fake images. Assign loss and accuracy to variable
    # FILL THIS IN
    gan_meta= gan.fit(noise2, labels_generator, epochs=epochs, batch_size=batch_size, shuffle=True).history
    gan_loss = [gan_meta['loss'][-1], gan_meta['accuracy'][-1]]

    # Save training status
    # Discriminator and model loss
    training_meta_data[e-1, 0] = d_loss[0]
    training_meta_data[e-1, 1] = gan_loss[0]

    # Discriminator and model accuracy
    training_meta_data[e-1, 2] = d_loss[1]
    training_meta_data[e-1, 3] = gan_loss[1]


    # If at sample interval, print training status and save samples
    if e % sample_interval == 0:
      
        # Print training status
        print("Epoch %d" %e)
        log_mesg = "%d: [Discriminaotr loss: %f, acc: %f]" % (e, d_loss[0], d_loss[1])
        log_mesg = "%s  [GAN loss: %f, acc: %f]" % (log_mesg, gan_loss[0], gan_loss[1])
        #print(log_mesg)
        
        # Plot images 
        r, c = 5, 5

        # Create images from the noise (predict the outcome of the noise)
        gen_imgs = generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow((gen_imgs[cnt].reshape(28, 28)), cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        plt.show()

NameError: name 'mnist' is not defined

In [ ]:
# Plot model loss vs epoch
#FILL THIS CODE BLOCK IN AND PRODUCE PLOT
plt.plot(dis_training_meta_data.history['loss'])
plt.plot(gan_meta.history['loss'])
plt.title('Discriminator and GAN model loss vs. Epoch')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Discrminator', 'GAN'], loc='upper right')
plt.show()

In [ ]:
# Plot accuracy vs epoch
#FILL THIS CODE BLOCK IN AND PRODUCE PLOT
plt.plot(dis_training_meta_data.history['accuracy'])
plt.plot(gan_meta.history['accuracy'])
plt.title('Discriminator and GAN model accuracy vs. Epoch')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Discrminator', 'GAN'], loc='upper right')
plt.show()

[4]. Compare and comment on the results of GAN with dropout and without dropout.


[5][a]. Comment on importance of hyper-parameter tuning


[6]. Answer the following questions:



1.   Why does the accuracy of the discriminator remain around 50%? Is this a good trait of the GAN? 

  ANS: As GAN is based on a minimax game based on an adversarial loss function, the generator and the discriminator are under a never-ending loop of oscillation. when the generator improves enough to fool the discriminator, the discriminator would have around 50% accuracy. At this point, the discriminator cannot give appropriate feedback and rather give random feedback which degrades the generator's performance again. After the generator's performance gets worse , the generator will learn the appropriate features again and this oscillation continues when both dicriminator and generator jointly search for equilibrium. So this is not a good trait of the GAN.


2.   How could this model be modified to produce cleaner (less noisy) images? 

  ANS: 1. modify the loss functions since loss functions provide guidance for model weights to follow in the vast state space, it is important that the functions represent the ultimate goals of optimization problems, such as LSGAN, the least-squares loss function. 2. GAN can also be improved with additional variants on architecture. The Neural Networks are variants of GAN resulting from the insights: Generator capable of having Variational Autoencoder (VAE) architecture and Discriminator having multiple outputs.

#Part 2: Generating samples using trained generator

In [ ]:
# Generate ten images from Gaussian noise using the trained generator from Part 1
# FILL THIS IN

# Re-scale generated images to lie in [0, 1]
# FILL THIS IN

In [ ]:
# Visualize generated noise
r, c = 2, 5
fig, axs = plt.subplots(r, c)
cnt = 0
for i in range(r):
    for j in range(c):
        axs[i,j].imshow((noise[cnt].reshape(10, 10)), cmap='gray')
        axs[i,j].axis('off')
        cnt += 1
plt.show()

In [ ]:
# Visualize generated samples
r, c = 2, 5
fig, axs = plt.subplots(r, c)
cnt = 0
for i in range(r):
    for j in range(c):
        axs[i,j].imshow((generated_images[cnt].reshape(28, 28)), cmap='gray')
        axs[i,j].axis('off')
        cnt += 1
plt.show()

#Part 3: Testing accuracy of generated images on ten samples

In [ ]:
# Load mnist classifier and generated images
mnist_classifier = load_model('mnist_classifier.h5')

In [1]:
# ASSIGN CLASSES
labels = []

# Convert integer labels to one-hot labels 
labels = keras.utils.np_utils.to_categorical(labels, num_classes=10)

# Show classifications
# FILL THIS IN 

# Evaluate accuracy
# FILL THIS IN 

NameError: name 'keras' is not defined